In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

import sys
from multiprocessing import Manager, Lock, Pool, cpu_count
import time

In [2]:
def progress(count, total, status=''):
    
    bar_len = 40
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '█' * filled_len + '░' * (bar_len - filled_len)

    sys.stdout.write(f'\r|{bar}| {percents}% ... {status}')
    sys.stdout.flush()

In [3]:
TRAINING_DATA = '/home/group2/brats/HGG/'
TRAINING_LABELS = '/home/group2/brats/survival_data.csv'
RADIOMIC_FEATURES = '/home/group2/marco/HGG_full.csv'

In [4]:
# survival label
def labelize(val):
    if 0 < val < 301:
        return 0
    if 301 <= val < 451:
        return 1 
    if val >= 451:
         return 2

def labelize2(val):
    if 0 < val < 365:
        return 0
    if val >= 365:
         return 1
        
df = pd.read_csv(TRAINING_LABELS)

In [5]:
X = pd.read_csv(RADIOMIC_FEATURES, index_col = 0)
X = X.replace(np.inf, np.nan)
X = X.dropna(axis=0)
X = X.set_index('sample')

In [6]:
X['cen_surv']=[labelize2(x) for x in X['surv']]

In [7]:
labels=X['cen_surv']
myX = X.iloc[:,:5556]

In [8]:
clf = RandomForestClassifier()
clf.fit(myX, labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [9]:
clf.predict(myX)

array([1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1])

In [10]:
pd.DataFrame(X['surv']).describe()

,surv
count,163.000000
mean,422.963190
std,349.683841
min,5.000000
25%,177.500000
50%,362.000000
75%,512.500000
max,1767.000000


In [11]:
def segment(array, parts):
    '''
    Function to segment the array in order to utilize multiprocessing
    '''
    
    avg = len(array) / parts
    last = 0.0

    while last < len(array):
        yield array[int(last):int(last + avg)]
        last += avg

In [14]:
def calc_mcc(exps):
    for exp in exps:
        x_train, x_test, y_train, y_test = train_test_split(myX, labels)
        clf = RandomForestClassifier(n_estimators=500)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        with Lock():
            mccs_man.append(matthews_corrcoef(y_test, y_pred))
            features_man.append(clf.feature_importances_)
            count.value += 1
            progress(count.value, num_exps, status='Calculating MCCs')

In [15]:
num_exps = 2000
test_size = 0.2

with Manager() as manager:
    # List of all features
    mccs_man = manager.list()
    count = manager.Value('i', 0)
    features_man = manager.list()
    
    t0 = time.time()

    with Pool(cpu_count()) as pool:
        tasks = [pool.apply_async(calc_mcc, args=(part,)) for part in segment([*range(num_exps)], cpu_count())]
        [task.get() for task in tasks]
        mccs = [x for x in mccs_man]
        top_features = [x for x in features_man]

    print(f' ... {round(time.time() - t0, 2)}s')

pd.DataFrame(mccs).describe()

|████████████████████████████████████████| 99.0% ... Calculating MCCs ... 154.11s


,0
count,2000.000000
mean,0.248834
std,0.128015
min,-0.098295
25%,0.169452
50%,0.269048
75%,0.360581
max,0.488493


In [18]:
importances = top_features
print(len(importances[0]))
tops = [[0, i] for i in range(5556)]

5556


In [19]:
for importance in importances:
    indices = np.argsort(importance)[::-1]
    for f in range(5556):
        tops[indices[f]][0] += 5556 - f

In [20]:
tops = [*sorted(tops, key=lambda el: el[0], reverse=True)][:10]

In [22]:
for couple in tops:
    print(myX.columns[couple[1]])

original_shape_Maximum2DDiameterRow_t1
gradient_gldm_DependenceVariance_t2
exponential_glszm_SmallAreaLowGrayLevelEmphasis_flair
original_shape_Maximum2DDiameterRow_t1ce
exponential_glszm_GrayLevelNonUniformity_t1
wavelet-LLH_firstorder_Skewness_t2
original_shape_Maximum2DDiameterRow_flair
wavelet-HLH_glszm_GrayLevelNonUniformityNormalized_t1
wavelet-LHH_glcm_Correlation_t2
log-sigma-3-0-mm-3D_glrlm_GrayLevelNonUniformity_t1
